# eda099_4
baseline再構築  

無印：フィルタなし  
仰角マスク30度

In [35]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.optimize as opt
import multiprocessing
pd.set_option('display.max_columns', 50)

In [36]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [37]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [38]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [39]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [40]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [41]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [42]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [43]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [44]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    return base_train, base_test, sample_sub, ground_truth

In [45]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

# baselineの再作成

In [46]:
def prepare_calc_baseline(df):
    light_speed = 299_792_458
    omega_e = 7.2921151467e-5
    
    # Corrected pseudorange according to data instructions
    df['correctedPrM'] = df['rawPrM'] + \
                         df['satClkBiasM'] - \
                         df['isrbM'] - \
                         df['ionoDelayM'] - \
                         df['tropoDelayM']
    
    # Time it took for signal to travel
    df['transmissionTimeSeconds'] = df['correctedPrM'] / light_speed
    
    # Compute true sat positions at arrival time
    df['xSatPosMRotated'] = \
        np.cos(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.sin(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['ySatPosMRotated'] = \
        - np.sin(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.cos(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['zSatPosMRotated'] = df['zSatPosM']
    
    # Uncertainty weight for the WLS method
    df['uncertaintyWeight'] = 1 / df['rawPrUncM']
    
    return df

In [47]:
def calc_baseline_point(df):

    def distance(sat_pos, x):
        sat_pos_diff = sat_pos.copy(deep=True)

        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff['uncertaintyWeight'] * \
                            (np.sqrt((sat_pos_diff['xSatPosMRotated']**2 + sat_pos_diff['ySatPosMRotated']**2 + sat_pos_diff['zSatPosMRotated']**2)) + \
                             x[3] - sat_pos_diff['correctedPrM'])

        return sat_pos_diff['d']

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)
    
    x0 = [0,0,0,0]
    opt_res = opt.least_squares(distance_fixed_satpos, x0)
    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x
    
    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    
    return wls_estimated_pos[0], wls_estimated_pos[1]

In [48]:
def get_derived_data(train_test, collection, phonename):
    derived = pd.read_csv(INPUT + f'/{train_test}/{collection}/{phonename}/{phonename}_derived.csv')
    raw = pd.read_csv(INPUT + f'/prep/gnss/{train_test}/{collection}/{phonename}/Raw.csv')
    status = pd.read_csv(INPUT + f'/prep/gnss/{train_test}/{collection}/{phonename}/Status.csv')
    
    delta_millis = derived['millisSinceGpsEpoch'] - derived['receivedSvTimeInGpsNanos'] / 1e6
    remove_signals = (delta_millis > 0) & (delta_millis < 300)
    derived = derived[~remove_signals].copy()
    
    # Assume we've loaded a dataframe from _GnssLog.txt for only lines beginning with "Raw", we denote this df_raw. Next, assume we've loaded a dataframe from _derived.csv. We denote this df_derived.

    # Create a new column in df_raw that corresponds to df_derived['MillisSinceGpsEpoch']
    raw['millisSinceGpsEpoch'] = np.floor( (raw['TimeNanos'] - raw['FullBiasNanos']) / 1000000.0).astype(int)
    
    # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
    raw_timestamps = raw['millisSinceGpsEpoch'].unique()
    derived_timestamps = derived['millisSinceGpsEpoch'].unique()

    # The timestamps in derived are one epoch ahead. We need to map each epoch
    # in derived to the prior one (in Raw).
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    derived['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], derived['millisSinceGpsEpoch'])))

    # Compute signal_type in df_raw.
    # Map from constellation id to frequencies and signals.
    CONSTEL_FREQ_TABLE = {
        0: {'UNKNOWN': (0, 999999999999)},
        1: {
            'GPS_L1': (1563000000, 1587000000),
            'GPS_L2': (1215000000, 1240000000),
            'GPS_L5': (1164000000, 1189000000)
        },
        3: {
            'GLO_G1': (1593000000, 1610000000),
            'GLO_G2': (1237000000, 1254000000)
        },
        4: {
            'QZS_J1': (1563000000, 1587000000),
            'QZS_J2': (1215000000, 1240000000),
            'QZS_J5': (1164000000, 1189000000)
        },
        5: {
            'BDS_B1C': (1569000000, 1583000000),
            'BDS_B1I': (1553000000, 1568990000),
            'BDS_B2A': (1164000000, 1189000000),
            'BDS_B2B': (1189000000, 1225000000)
        },
        6: {
            'GAL_E1': (1559000000, 1591000000),
            'GAL_E5A': (1164000000, 1189000000),
            'GAL_E5B': (1189000000, 1218000000),
            'GAL_E6': (1258000000, 1300000000)
        },
        7: {
            'IRN_S': (2472000000, 2512000000),
            'IRN_L5': (1164000000, 1189000000)
        },
    }

    def SignalTypeFromConstellationAndFequency(constel, freq_hz):
        'Returns the signal type as a string for the given constellation and frequency.'
        freqs = CONSTEL_FREQ_TABLE.get(constel, {})
        for id_freq_range in freqs.items():
            rng = id_freq_range[1]
            if rng[0] <= freq_hz <= rng[1]:
                return id_freq_range[0]
        return 'UNKNOWN'

    signal_types = itertools.chain(*[c.keys() for c in CONSTEL_FREQ_TABLE.values()])
    sig_type_cat = pd.api.types.CategoricalDtype(categories=signal_types)
    raw['signalType'] = raw.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)

    # Fix QZS Svids issue. 

    # The SVID of any QZS sat in derived may be changed. Since it may be a many to one relationship, we'll need to adjust the values in Raw.
    new_to_old = {1:(183, 193), 2:(184, 194, 196), 3:(187, 189, 197, 199), 4:(185, 195, 200)}
    # Maps original svid to new svid for only ConstellationType=4.
    old_to_new={}
    for new_svid, old_svids in new_to_old.items():
        for s in old_svids:
            old_to_new[s] = new_svid
    raw['svid'] = raw.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    del raw['collectionName']
    del raw['phoneName']
    
    derived = derived.merge(raw, on=['millisSinceGpsEpoch', 'svid', 'signalType'], how='left')
    
    # status
    status['millisSinceGpsEpoch'] = status['UnixTimeMillis'] - 315964800000 + 18000
    status['svid'] = status.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    status['signalType'] = status.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)
    del status['collectionName']
    del status['phoneName']
    status = status.drop_duplicates(subset=['svid', 'signalType', 'millisSinceGpsEpoch'])
    
    sv_sig = derived[['svid', 'signalType']].drop_duplicates()
    output_df = pd.DataFrame()
    for svid, signal in zip(sv_sig['svid'], sv_sig['signalType']):
        derived_tmp = derived[(derived['svid']==svid)&(derived['signalType']==signal)].copy()
        status_tmp = status[(status['svid']==svid)&(status['signalType']==signal)].copy()
        del status_tmp['svid']
        del status_tmp['signalType']
        output_tmp = pd.merge_asof(derived_tmp, status_tmp, on='millisSinceGpsEpoch', direction='nearest')
        output_df = output_df.append(output_tmp)
    
    return output_df

In [49]:
def calc_baseline(args):
    s_th = 10
    
    phone, df = args
    collection = phone.split('_')[0]
    phonename = phone.split('_')[1]
    derived = get_derived_data('train', collection, phonename)
    derived = prepare_calc_baseline(derived)
    
    #derived = derived[~(derived['MultipathIndicator']==1)].copy()
    derived = derived[~(derived['ElevationDegrees']<=30)].copy()
    #derived = derived[~(derived['UsedInFix']==0)].copy()
    
    idx = list(df.index)
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    unc_mean_list = []
    unc_max_list = []
    
    for j,i in enumerate(idx):
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[(derived['millisSinceGpsEpoch']>=s-s_th)&(derived['millisSinceGpsEpoch']<=s+s_th)].copy()
        n = tmp['svid'].nunique()
        s_list.append(s)
        n_list.append(n)
        
        if n < 4:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
            unc_mean_list.append(np.nan)
            unc_max_list.append(np.nan)        
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
            unc_mean_list.append(tmp['uncertaintyWeight'].mean())
            unc_max_list.append(tmp['uncertaintyWeight'].max())
    
    output_df = pd.DataFrame()
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    output_df['unc_mean'] = unc_mean_list
    output_df['unc_max'] = unc_max_list
    output_df['collectionName'] = collection
    output_df['phoneName'] = phonename
    output_df['phone'] = phone
    
    return output_df

In [50]:
def calc_baseline_test(args):
    s_th = 10
    
    phone, df = args
    collection = phone.split('_')[0]
    phonename = phone.split('_')[1]
    derived = get_derived_data('test', collection, phonename)
    derived = prepare_calc_baseline(derived)
    
    #derived = derived[~(derived['MultipathIndicator']==1)].copy()
    derived = derived[~(derived['ElevationDegrees']<=30)].copy()
    #derived = derived[~(derived['UsedInFix']==0)].copy()
    
    idx = list(df.index)
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    unc_mean_list = []
    unc_max_list = []
    
    for j,i in enumerate(idx):
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[(derived['millisSinceGpsEpoch']>=s-s_th)&(derived['millisSinceGpsEpoch']<=s+s_th)].copy()
        n = tmp['svid'].nunique()
        s_list.append(s)
        n_list.append(n)
        
        if n < 4:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
            unc_mean_list.append(np.nan)
            unc_max_list.append(np.nan)        
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
            unc_mean_list.append(tmp['uncertaintyWeight'].mean())
            unc_max_list.append(tmp['uncertaintyWeight'].max())
    
    output_df = pd.DataFrame()
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    output_df['unc_mean'] = unc_mean_list
    output_df['unc_max'] = unc_max_list
    output_df['collectionName'] = collection
    output_df['phoneName'] = phonename
    output_df['phone'] = phone
    
    return output_df

In [17]:
#df = get_derived_data('train', '2021-04-28-US-MTV-1', 'Pixel5')

In [51]:
train, test, sub, gt = get_data()

In [19]:
target = ['2021-04-22-US-SJC-1_Pixel4', '2021-04-22-US-SJC-1_SamsungS20Ultra',
          '2021-04-28-US-SJC-1_Pixel4', '2021-04-28-US-SJC-1_SamsungS20Ultra',
          '2021-04-29-US-SJC-2_Pixel4', '2021-04-29-US-SJC-2_SamsungS20Ultra',
          '2021-04-28-US-MTV-1_Pixel4', '2021-04-28-US-MTV-1_Pixel5', '2021-04-28-US-MTV-1_SamsungS20Ultra',
          '2021-04-29-US-MTV-1_Pixel4', '2021-04-29-US-MTV-1_Pixel5', '2021-04-29-US-MTV-1_SamsungS20Ultra']
train = train[train['phone'].isin(target)].copy()

In [20]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_baseline, train.groupby('phone'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
result = pd.concat(dfs)

0it [00:00, ?it/s]

In [21]:
train_tmp = train[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
train_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_bl', 'lngDeg_bl']
result = result.merge(train_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [22]:
gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']
gt_tmp = gt[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']].copy()
gt_tmp.columns = ['phone', 'millisSinceGpsEpoch', 'latDeg_gt', 'lngDeg_gt']
result = result.merge(gt_tmp, on=['phone', 'millisSinceGpsEpoch'], how='left')

In [23]:
result['rb_bl_err'] = calc_haversine(result['latDeg_bl'], result['lngDeg_bl'], result['latDeg'], result['lngDeg'])
result['rb_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg'], result['lngDeg'])
result['bl_gt_err'] = calc_haversine(result['latDeg_gt'], result['lngDeg_gt'], result['latDeg_bl'], result['lngDeg_bl'])

In [24]:
for phone in result['phone'].unique():
    tmp = result[result['phone']==phone].copy()
    null_rate = tmp['latDeg'].isnull().sum() / len(tmp)
    print(phone, null_rate)

2021-04-29-US-MTV-1_SamsungS20Ultra 0.0006027727546714888
2021-04-29-US-MTV-1_Pixel4 0.0006242197253433209
2021-04-29-US-MTV-1_Pixel5 0.0006265664160401002
2021-04-28-US-MTV-1_SamsungS20Ultra 0.0005047955577990914
2021-04-28-US-MTV-1_Pixel5 0.0005037783375314861
2021-04-28-US-SJC-1_Pixel4 0.0014895729890764648
2021-04-28-US-MTV-1_Pixel4 0.0005002501250625312
2021-04-29-US-SJC-2_SamsungS20Ultra 0.0004219409282700422
2021-04-28-US-SJC-1_SamsungS20Ultra 0.0009601536245799327
2021-04-29-US-SJC-2_Pixel4 0.0012875536480686696
2021-04-22-US-SJC-1_Pixel4 0.0010380622837370243
2021-04-22-US-SJC-1_SamsungS20Ultra 0.00035385704175513094


In [25]:
result_grouped = result.dropna().groupby('phone')[['rb_bl_err', 'rb_gt_err', 'bl_gt_err']].agg([percentile50, percentile95]).reset_index()
result_grouped.columns = ['phone', 'rb_bl_err_p50', 'rb_bl_err_p95', 'rb_gt_err_p50', 'rb_gt_err_p95', 'bl_gt_err_p50', 'bl_gt_err_p95']
result_grouped['rb_score'] = (result_grouped['rb_gt_err_p95'] + result_grouped['rb_gt_err_p50']) / 2
result_grouped['bl_score'] = (result_grouped['bl_gt_err_p95'] + result_grouped['bl_gt_err_p50']) / 2
result_grouped

,phone,rb_bl_err_p50,rb_bl_err_p95,rb_gt_err_p50,rb_gt_err_p95,bl_gt_err_p50,bl_gt_err_p95,rb_score,bl_score
0,2021-04-22-US-SJC-1_Pixel4,2.339846,21.527008,7.432139,39.530871,6.996956,42.133024,23.481505,24.564990
1,2021-04-22-US-SJC-1_SamsungS20Ultra,1.328419,13.398181,7.394012,26.318340,7.406780,28.133754,16.856176,17.770267
2,2021-04-28-US-MTV-1_Pixel4,2.020834,6.894106,2.382116,9.043376,2.164321,7.365087,5.712746,4.764704
3,2021-04-28-US-MTV-1_Pixel5,1.782634,6.631693,2.015375,6.979645,1.650982,3.956857,4.497510,2.803919
4,2021-04-28-US-MTV-1_SamsungS20Ultra,1.097228,4.836240,3.827346,13.032359,3.397275,11.083685,8.429852,7.240480
5,2021-04-28-US-SJC-1_Pixel4,1.707773,10.870424,3.985492,23.127362,4.170432,24.495735,13.556427,14.333083
6,2021-04-28-US-SJC-1_SamsungS20Ultra,1.593440,19.273424,6.650340,33.918759,5.727631,34.629760,20.284550,20.178696
7,2021-04-29-US-MTV-1_Pixel4,4.751820,23.455397,8.081113,26.382105,6.026172,16.005119,17.231609,11.015645
8,2021-04-29-US-MTV-1_Pixel5,7.406743,54.598179,9.562397,58.630897,6.595427,15.593213,34.096647,11.094320
9,2021-04-29-US-MTV-1_SamsungS20Ultra,0.881306,7.167627,6.510894,14.321618,6.344816,13.739767,10.416256,10.042291


In [26]:
result.to_csv(OUTPUT + '/result.csv', index=False)
result_grouped.to_csv(OUTPUT + '/result_grouped.csv', index=False)

In [27]:
result_grouped[['rb_score', 'bl_score']].mean()

rb_score    15.818338
bl_score    13.617523
dtype: float64

In [28]:
target = ['2021-04-22-US-SJC-1_Pixel4', '2021-04-22-US-SJC-1_SamsungS20Ultra',
          '2021-04-28-US-SJC-1_Pixel4', '2021-04-28-US-SJC-1_SamsungS20Ultra',
          '2021-04-29-US-SJC-2_Pixel4', '2021-04-29-US-SJC-2_SamsungS20Ultra']
result_grouped[result_grouped['phone'].isin(target)][['rb_score', 'bl_score']].mean()

rb_score    18.239239
bl_score    19.408153
dtype: float64

In [29]:
target = ['2021-04-28-US-MTV-1_Pixel4', '2021-04-28-US-MTV-1_Pixel5', '2021-04-28-US-MTV-1_SamsungS20Ultra',
          '2021-04-29-US-MTV-1_Pixel4', '2021-04-29-US-MTV-1_Pixel5', '2021-04-29-US-MTV-1_SamsungS20Ultra']
result_grouped[result_grouped['phone'].isin(target)][['rb_score', 'bl_score']].mean()

rb_score    13.397437
bl_score     7.826893
dtype: float64

In [30]:
result

,millisSinceGpsEpoch,latDeg,lngDeg,n,unc_mean,unc_max,collectionName,phoneName,phone,latDeg_bl,lngDeg_bl,latDeg_gt,lngDeg_gt,rb_bl_err,rb_gt_err,bl_gt_err
0,1303768905000,37.395777,-122.103030,13,0.173563,0.476417,2021-04-29-US-MTV-1,SamsungS20Ultra,2021-04-29-US-MTV-1_SamsungS20Ultra,37.395792,-122.103028,37.395822,-122.102928,1.732240,10.299507,9.411230
1,1303768906000,37.395782,-122.102990,13,0.177939,0.476417,2021-04-29-US-MTV-1,SamsungS20Ultra,2021-04-29-US-MTV-1_SamsungS20Ultra,37.395796,-122.102999,37.395822,-122.102928,1.683678,7.053847,6.947362
2,1303768907000,37.395808,-122.102987,13,0.177601,0.476417,2021-04-29-US-MTV-1,SamsungS20Ultra,2021-04-29-US-MTV-1_SamsungS20Ultra,37.395816,-122.102995,37.395822,-122.102928,1.155180,5.392692,5.922113
3,1303768908000,37.395820,-122.102990,13,0.183331,0.555864,2021-04-29-US-MTV-1,SamsungS20Ultra,2021-04-29-US-MTV-1_SamsungS20Ultra,37.395825,-122.102993,37.395822,-122.102928,0.636093,5.499344,5.728542
4,1303768909000,37.395815,-122.103008,13,0.186831,0.555864,2021-04-29-US-MTV-1,SamsungS20Ultra,2021-04-29-US-MTV-1_SamsungS20Ultra,37.395821,-122.103006,37.395822,-122.102928,0.633446,7.069790,6.922120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25330,1303163398000,37.334594,-121.899426,12,0.224447,0.555864,2021-04-22-US-SJC-1,SamsungS20Ultra,2021-04-22-US-SJC-1_SamsungS20Ultra,37.334605,-121.899437,37.334600,-121.899424,1.546772,0.701390,1.202934
25331,1303163399000,37.334612,-121.899448,12,0.233363,0.555864,2021-04-22-US-SJC-1,SamsungS20Ultra,2021-04-22-US-SJC-1_SamsungS20Ultra,37.334603,-121.899428,37.334600,-121.899424,1.945980,2.429544,0.503375
25332,1303163400000,37.334623,-121.899427,12,0.235041,0.555864,2021-04-22-US-SJC-1,SamsungS20Ultra,2021-04-22-US-SJC-1_SamsungS20Ultra,37.334622,-121.899414,37.334600,-121.899424,1.175612,2.604981,2.556369
25333,1303163401000,37.334630,-121.899437,12,0.212936,0.555864,2021-04-22-US-SJC-1,SamsungS20Ultra,2021-04-22-US-SJC-1_SamsungS20Ultra,37.334623,-121.899409,37.334600,-121.899424,2.660964,3.535545,2.911981


# test

In [52]:
target = ['2021-04-22-US-SJC-2_SamsungS20Ultra', '2021-04-29-US-SJC-3_Pixel4', '2021-04-29-US-SJC-3_SamsungS20Ultra']
test = test[test['phone'].isin(target)].copy()

In [53]:
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(calc_baseline_test, test.groupby('phone'))
    dfs = tqdm(dfs)
    dfs = list(dfs)
result = pd.concat(dfs)

0it [00:00, ?it/s]

In [54]:
result.to_csv(OUTPUT + '/result_test.csv', index=False)

In [55]:
for phone in result['phone'].unique():
    tmp = result[result['phone']==phone].copy()
    null_rate = tmp['latDeg'].isnull().sum() / len(tmp)
    print(phone, null_rate)

2021-04-29-US-SJC-3_Pixel4 0.0015159171298635675
2021-04-29-US-SJC-3_SamsungS20Ultra 0.00048520135856380397
2021-04-22-US-SJC-2_SamsungS20Ultra 0.0008605851979345956
